# Data Preparation

### 1- Dealing with Outliers

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#Import Ames House Dataset
df= pd.read_csv('../input/house-price-prediction-dataset/Housing_Price_Data.csv')

In [ ]:
df.head()

In [ ]:
#Our objective is to predict the SalePrice based on several Features, 
# Correlation overview:
df.corr()['SalePrice'].sort_values()

In [ ]:
sns.scatterplot(data=df, x='Overall Qual', y='SalePrice')
plt.axhline(y=200000,color='r')

In [ ]:
df[(df['Overall Qual']>8) &(df['SalePrice']<200000)][['SalePrice', 'Overall Qual']]

In [ ]:
sns.scatterplot(x='Gr Liv Area', y='SalePrice', data=df)
plt.axhline(y=200000, color='r')
plt.axvline(x=4000, color='r')

In [ ]:
df[(df['Gr Liv Area']>4000) & (df['SalePrice']<400000)][['SalePrice', 'Gr Liv Area']]

In [ ]:
#Remove the outliers:
index_drop=df[(df['Gr Liv Area']>4000) & (df['SalePrice']<400000)].index
df=df.drop(index_drop, axis=0)

In [ ]:
sns.scatterplot(x='Gr Liv Area', y='SalePrice', data=df)
plt.axhline(y=200000, color='r')
plt.axvline(x=4000, color='r')

In [ ]:
sns.scatterplot(x='Overall Qual', y='SalePrice', data=df)
plt.axhline(y=200000,color='r')

In [ ]:
sns.boxplot(x='Overall Qual', y='SalePrice', data=df)

## 2-Dealing with Missing Data

In [ ]:
df.head()

In [ ]:
df.info()

### Fill/Keep/Drop?

In [ ]:
#Removing the PID (We already have an index, so we don't need PID unique identifier. 
#becuase it doesn't have any information and can't help our learning models)

df= df.drop('PID', axis=1)

### A- How Much Data is Missing?

In [ ]:
df.isnull()

In [ ]:
#How many missing data is there in each features?
df.isnull().sum()

In [ ]:
#The percent of missing data in any feature:
100*(df.isnull().sum()/len(df))

In [ ]:
#Make a Function to calculate the percent of missing data in each columns (feature) and then sort it
def missing_percent(df):
    nan_percent= 100*(df.isnull().sum()/len(df))
    nan_percent= nan_percent[nan_percent>0].sort_values()
    return nan_percent

In [ ]:
nan_percent= missing_percent(df)

In [ ]:
nan_percent

In [ ]:
#plot the feature with missing indicating the percent of missing data
plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent)
plt.xticks(rotation=90)

### B- Working base on Rows Missing Data

In [ ]:
#every Feature with missing data must be checked!
#We choose a threshold of 1%. It means, if there is less than 1% of a feature are missing,
#then we will consider just dropping that rows

plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent)
plt.xticks(rotation=90)

#Set 1% threshold:
plt.ylim(0,1)

In [ ]:
nan_percent[nan_percent<1]

In [ ]:
nan_percent[nan_percent<1].index

In [ ]:
100/len(df)
#It shows that, Feature with just one missing rows has this percent value of missing data

In [ ]:
df[df['Electrical'].isnull()]

In [ ]:
df[df['Garage Area'].isnull()]

### Dropping Rows:

In [ ]:
df= df.dropna(axis=0, subset=['Electrical', 'Garage Area'])

In [ ]:
nan_percent= missing_percent(df)

plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent)
plt.xticks(rotation=90)
plt.ylim(0,1)

### Filling the missing values:

In [ ]:
#Features related to Basement:

In [ ]:
df[df['Total Bsmt SF'].isnull()]

In [ ]:
df[df['Bsmt Half Bath'].isnull()]

In [ ]:
df[df['Bsmt Full Bath'].isnull()]

In [ ]:
#After checking the data documentation,
#it shows that missing value (two rows) in Basement Features are becouse of there is no basement in these rows
#Decision: Filling in data based on column: numerical basement & string descriptive:

#Numerical Columns fill with 0:
bsmt_num_cols= ['BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF','Total Bsmt SF' ,'Bsmt Full Bath', 'Bsmt Half Bath']
df[bsmt_num_cols]=df[bsmt_num_cols].fillna(0)

#String Columns fill with None:
bsmt_str_cols= ['Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2']
df[bsmt_str_cols]= df[bsmt_str_cols].fillna('None')

In [ ]:
nan_percent= missing_percent(df)

plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent)
plt.xticks(rotation=90)
plt.ylim(0,1)

**Mas Vnr Features:**

Based on the Dataset Document File, missing values for 'Mas Vnr Type' and 'Mas Vnr Area' means the house doesn't have any mansonry veneer. so, we decide to fill the missing value as below: 

In [ ]:
df["Mas Vnr Type"]= df["Mas Vnr Type"].fillna("None")
df["Mas Vnr Area"]= df["Mas Vnr Area"].fillna(0)

In [ ]:
nan_percent= missing_percent(df)

plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent)
plt.xticks(rotation=90)


### Garage Columns:
**Based on the dataset documentation, NaN in Garage Columns seems to indicate no garage.**

**Decision: Fill with 'None' or 0**

In [ ]:
df[['Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Garage Qual', 'Garage Cond']]

In [ ]:
#Filling the missing Value:
Gar_str_cols= ['Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond']
df[Gar_str_cols]=df[Gar_str_cols].fillna('None')

df['Garage Yr Blt']=df['Garage Yr Blt'].fillna(0)

In [ ]:
nan_percent= missing_percent(df)

plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent)
plt.xticks(rotation=90)


### C-Working based on Columns Missing Data

In [ ]:
nan_percent.index

In [ ]:
df[['Lot Frontage', 'Fireplace Qu', 'Fence', 'Alley', 'Misc Feature',
       'Pool QC']]

**Remove the Columns with more than 80% missing values**

In [ ]:
df= df.drop(['Fence', 'Alley', 'Misc Feature','Pool QC'], axis=1)

In [ ]:
nan_percent= missing_percent(df)

plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent)
plt.xticks(rotation=90)

In [ ]:
#Filling in Fireplace Quality based on dataset documentation:
df['Fireplace Qu']= df['Fireplace Qu'].fillna('None')

In [ ]:
nan_percent= missing_percent(df)

plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent)
plt.xticks(rotation=90)

### D- Imputation of Missing Data

**Columns: Lot Frontage**

**We assume that the Lot Frontage is related to what a Neighborhood a house is in**

In [ ]:
df['Neighborhood'].unique()

In [ ]:
plt.figure(figsize=(8,12))
sns.boxplot(data=df, x='Lot Frontage', y='Neighborhood')

In [ ]:
#Impute missing data based on other columns:

df.groupby('Neighborhood')['Lot Frontage']

In [ ]:
df.groupby('Neighborhood')['Lot Frontage'].mean()

In [ ]:
df.groupby('Neighborhood')['Lot Frontage'].transform(lambda val: val.fillna(val.mean()))

In [ ]:
df['Lot Frontage']=df.groupby('Neighborhood')['Lot Frontage'].transform(lambda val: val.fillna(val.mean()))

In [ ]:
nan_percent= missing_percent(df)

plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent)
plt.xticks(rotation=90)

In [ ]:
df['Lot Frontage']= df['Lot Frontage'].fillna(0)

In [ ]:
nan_percent= missing_percent(df)


In [ ]:
nan_percent

**We don't have any Missing Data**

## 3-Dealing with Categorical Data

#### A- Numerical Columns to Categorical
We need to be careful when it comes to encoding categorical as numbers. We want to make sure that the numerical relationship makes sense for model. For example, the encoding MSSubClass is essentially just a code per class

In [ ]:
df['MS SubClass']

In [ ]:
df.info()

In [ ]:
df['MS SubClass'].unique()

In [ ]:
#Convert to String:
df['MS SubClass']= df['MS SubClass'].apply(str)

In [ ]:

df.info()
#or: df['MS SubClass'].dtype

### B- Creating Dummy Variables

In [ ]:
df.select_dtypes(include='object')

In [ ]:
df_num= df.select_dtypes(exclude='object')
df_obj= df.select_dtypes(include='object')

In [ ]:
df_num.info()

In [ ]:
df_obj.info()

In [ ]:
# Converting:
df_obj= pd.get_dummies(df_obj, drop_first=True)

In [ ]:
df_obj.shape

In [ ]:
Final_df= pd.concat([df_num, df_obj], axis=1)

In [ ]:
Final_df.head()

#### Now, the Dataset is Ready for any Machine Learing Model & Analysis